In [1]:
# Compare the skill of the model forecast using the depth data at the resolution of the DEM.

# Library imports.
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat
import os
import rasterio
import geojson

# Local imports.
from rasopt.Utilities import utils, land_cover_utils

In [40]:
# Load in the files.
# Directory to files. 
file_dir = r"C:\Users\ay434\Box\Research\Flood_Sim_Materials\BayesOpt_Paper\Data\Roughness_Output"

# Ground truth.
gt_fname = "Secchia_Panaro.p23_GT.hdf"
gt_fp = os.path.join(file_dir, gt_fname)

# Calibrated model.
cal_fname = "Secchia_Panaro.p23_c4s3.hdf"
cal_fp = os.path.join(file_dir, cal_fname)
cal_id = os.path.splitext(cal_fname)[0].split('_')[-1]

# HDF paths.
# Cell coordinate path.
cell_coord_path = 'Geometry/2D Flow Areas/Secchia_Panaro/Cells Center Coordinate'

# Water depths path.
depth_path = ('Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/'
              '2D Flow Areas/Secchia_Panaro/Depth')

# Manning's n calibration table path.
cal_table_path = "Geometry/Land Cover (Manning's n)/Calibration Table"

# Cell facepoint index path.
cell_facepoint_idx_path = 'Geometry/2D Flow Areas/Secchia_Panaro/Cells FacePoint Indexes'

# Facepoint coordinate path.
facepoint_coord_path = 'Geometry/2D Flow Areas/Secchia_Panaro/FacePoints Coordinate'

In [41]:
# Extract ground truth depths.
gt_depths = utils.extract_depths(gt_fp, depth_path, cell_coord_path)

# Extract calibrated depths.
cal_depths = utils.extract_depths(cal_fp, depth_path, cell_coord_path)

# Number of cells and time steps.
ncell, ncol = gt_depths.shape
Nt = ncol - 2

In [42]:
# Count the number of timesteps where the inundation prediction and truth did not match. 
inun_cut = 0.1
gt_inun = np.zeros((1,ncell)) # Count how many time steps each cell is inundated.
gt_and_notcal = np.zeros((1,ncell))
notgt_and_cal = np.zeros((1,ncell))
gt_and_cal = np.zeros((1,ncell))
gt_eq_cal = np.zeros((1,ncell))
for i in range(2,ncol):
    # Depths from a single time step.
    Dgt = gt_depths.iloc[:,i].to_numpy()
    Dcal = cal_depths.iloc[:,i].to_numpy()
    
    # Set depths above inun_cut to 1 and below inun_cut to 0.
    Dgt[Dgt >= inun_cut] = 1
    Dgt[Dgt < inun_cut] = 0
    gt_inun += Dgt
    gt_bool = np.array(Dgt, dtype=bool)
    Dcal[Dcal >= inun_cut] = 1
    Dcal[Dcal < inun_cut] = 0
    cal_bool = np.array(Dcal, dtype=bool)
    
    # Compute logical values.
    gt_and_cal += np.logical_and(gt_bool, cal_bool).astype(float)
    gt_and_notcal += np.logical_and(gt_bool, ~cal_bool).astype(float)
    notgt_and_cal += np.logical_and(~gt_bool, cal_bool).astype(float)
    gt_eq_cal += gt_bool == cal_bool
    
# Normalize the gt_eq_cal by the number of time steps.
gt_eq_cal /= Nt

In [43]:
# Create depth geojson for the logical arrays.
# gt_and_cal_gjson = utils.depth_geojson(gt_fp, cell_facepoint_idx_path, facepoint_coord_path, 
#                                      depth_path, cell_coord_path, depths=gt_and_cal[0])
# gt_and_notcal_gjson = utils.depth_geojson(gt_fp, cell_facepoint_idx_path, facepoint_coord_path, 
#                                      depth_path, cell_coord_path, depths=gt_and_notcal[0])
# notgt_and_cal_gjson = utils.depth_geojson(gt_fp, cell_facepoint_idx_path, facepoint_coord_path, 
#                                      depth_path, cell_coord_path, depths=notgt_and_cal[0])
gt_eq_cal_gjson = utils.depth_geojson(gt_fp, cell_facepoint_idx_path, facepoint_coord_path, 
                                     depth_path, cell_coord_path, depths=gt_eq_cal[0])

# Save geojsons as a rasters.
save_dir = r"C:\Users\ay434\Box\Research\Flood_Sim_Materials\BayesOpt_Paper\Data\Roughness_Output\Logical_Inundation_Rasters"
property_name = "Depth"
cell_width_X = 20
cell_width_Y = 20

# # GT and Cal
# raster_fname = f"gt_and_cal_{cal_id}.tif"
# utils.rasterize_geojson(gt_and_cal_gjson, property_name, save_dir, raster_fname, cell_width_X, cell_width_Y)

# # GT and Not Cal
# raster_fname = f"gt_and_notcal_{cal_id}.tif"
# utils.rasterize_geojson(gt_and_notcal_gjson, property_name, save_dir, raster_fname, cell_width_X, cell_width_Y)

# # Not GT and Cal
# raster_fname = f"notgt_and_cal_{cal_id}.tif"
# utils.rasterize_geojson(notgt_and_cal_gjson, property_name, save_dir, raster_fname, cell_width_X, cell_width_Y)

# GT equal Cal
raster_fname = f"gt_eq_cal_{cal_id}.tif"
utils.rasterize_geojson(gt_eq_cal_gjson, property_name, save_dir, raster_fname, cell_width_X, cell_width_Y)

C:\Users\ay434\Box\Research\Flood_Sim_Materials\BayesOpt_Paper\Data\Roughness_Output\Secchia_Panaro.p23_GT.hdf
gdal_rasterize -at -l tmp_gjson -a Depth -tr 20 20 -a_nodata -999.0 -te 652569.026634 4945866.106266 682199.040362 4970716.987691 -ot Float32 -of GTiff C:\Users\ay434\Box\Research\Flood_Sim_Materials\BayesOpt_Paper\Data\Roughness_Output\Logical_Inundation_Rasters\tmp_gjson.geojson C:\Users\ay434\Box\Research\Flood_Sim_Materials\BayesOpt_Paper\Data\Roughness_Output\Logical_Inundation_Rasters\gt_eq_cal_c4s3.tif


(array([[-999., -999., -999., ..., -999., -999., -999.],
        [-999., -999., -999., ..., -999., -999., -999.],
        [-999., -999., -999., ..., -999., -999., -999.],
        ...,
        [-999., -999., -999., ..., -999., -999., -999.],
        [-999., -999., -999., ..., -999., -999., -999.],
        [-999., -999., -999., ..., -999., -999., -999.]], dtype=float32),
 'C:\\Users\\ay434\\Box\\Research\\Flood_Sim_Materials\\BayesOpt_Paper\\Data\\Roughness_Output\\Logical_Inundation_Rasters\\gt_eq_cal_c4s3.tif')